In [1]:
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()
import json

import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from random import shuffle
import pandas as pd
import random

import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.layers import Input, Dense, Lambda
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

from rdkit import Chem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import Draw

import sys
sys.path.insert(0, '../')
from scripts import build_vae, decode_smiles

/home/wesleybeckner/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#for getting cation data
properties = ['density', 'cpt', 'viscosity', 'thermal_conductivity',
              'melting_point']
props = properties
devmodel = salty.aggregate_data(props, merge='Union')
devmodel.Data['smiles_string'] = devmodel.Data['smiles-cation']
cations = devmodel.Data['smiles_string'].drop_duplicates()
cations = cations.reset_index(drop=True)

In [12]:
vae_models = ['Bootstrap_250k_Cation_1.h5', 'Bootstrap_1Mil_Cation_1.h5',
    '1Mil_GDB17.h5', '1Mil_GDB17_split_500k.h5', '1Mil_GDB17_split_500k_cation_500k.h5']

f = open("../data/1mil_GDB17.json","r")
char_to_index = json.loads(f.read())
autoencoder = build_vae()
smi = 'CCCC[n+]1ccc(cc1)C'
autoencoder.load_weights("../data/{}".format(vae_models[-1]))

In [6]:
def generate_structures(vae, smi, char_to_index, limit=1e4):
    rdkit_mols = []
    temps = []
    iterations = []
    iteration = limit_counter = 0
    while True:
        iteration += 1
        limit_counter += 1
        t = random.random()*2
        candidate = decode_smiles(vae, smi, char_to_index, temp=t).split(" ")[0]
        try:
            sampled = Chem.MolFromSmiles(candidate)
            if candidate not in rdkit_mols:
                temps.append(t)
                iterations.append(iteration)
                rdkit_mols.append(candidate) 
                limit_counter = 0
        except:
            pass
        if limit_counter > limit:
            break
    df = pd.DataFrame([rdkit_mols,temps,iterations]).T
    df.columns = ['smiles', 'temperature', 'iteration']
    return df

In [ ]:
generate_structures(autoencoder, smi, char_to_index, limit=1e3)

../scripts/vae.py:77: RuntimeWarning: invalid value encountered in true_divide
  dist = np.exp(a)/np.sum(np.exp(a))
../scripts/vae.py:79: RuntimeWarning: invalid value encountered in less
  return np.random.choice(choices, p=dist)
